# Introduction

This notebook demonstrates how to use partiiton strategy to solve larger scale route optimization problem. The rationale for partitioning is that usually an optimization problem could be hard to solve given the NP-hard nature for most of the optimization problems. To trade-off the result optimality and running time, one can partition the big problem into many smaller problems, then solve each smaller problem, and finially combine all results as the final result. The whole pipeline is illustrated by the below figure.

<img src=../docs/media/pipeline.png width="90%" />

There are 4 main steps in the pipeline:
1.  Reduce: It will try to assign some of the orders to truck routes in a heuristic way. The remaining unscheduled order will be passed to the later steps for optimization. This step is optional, namely, one can bypass this step but let optimizer search solution for all orders. However, reducing the search space by  heuristic can significantly reduce the search space. This will make it easier for the oprimization solver to find a good solution.  
2.  Partition: This is core step to partition the big problem into smaller problems. 
3.  Solve: This step is to solve individual small problem using whatever optimization solver.
4.  Merge: This final step is to combine all results from each small problem.




# 1.0 Load libraries

We use Azure ML pipeline for the implementation. Specifically, the partitioning step is done by the PrallelRunStep in Azure ML SDK.

In [1]:
# import required libraries
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction

# 1.1 Setup some environment
## 1.1.1 Load variables

Some parameters are managed by environment variables.To specify your values, create a .env file in the root folder of the repository and set the values for the following parameters.

In [2]:
load_dotenv()

ws_name = os.environ['AML_WORKSPACE_NAME']
subscription_id = os.environ['AML_SUBSCRIPTION_ID']
resource_group = os.environ['AML_RESOURCE_GROUP']


print('---- Check Azure setting ----')
print(f'AML Workspace name       : {ws_name}')
print(f'Subscription ID          : {subscription_id}')
print(f'Resource group           : {resource_group}')

---- Check Azure setting ----
AML Workspace name       : amldemo
Subscription ID          : e4eda206-7aff-4e54-8f55-1e60f0e64093
Resource group           : aml-demo-rg


## 1.1.2 Azure authentication and Load Azure ML Workspace

We are using DefaultAzureCredential to get access to workspace.

DefaultAzureCredential should be capable of handling most Azure SDK authentication scenarios.

Reference for more available credentials if it does not work for you: configure credential example, azure-identity reference doc.

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": ws_name,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: C:\Users\zhianhe\Demo\.azureml\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000002A26D6C78E0>,
         subscription_id=e4eda206-7aff-4e54-8f55-1e60f0e64093,
         resource_group_name=aml-demo-rg,
         workspace_name=amldemo)


## 1.1.3 Get Compute Cluster

Read the compute name from the environment varibale. If it doest not exist in the Azure ML workspace, a new compute target will be created.

In [5]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name for the optimization job
cpu_compute_target = "op-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="Standard_E4ds_v4", min_instances=0, max_instances=10
    )
    ml_client.compute.begin_create_or_update(compute).result()

## 1.1.4 Create AML Environemnt and Run Configuration

In [ ]:
# environment
env_name = 'op-env'

try:
    env = ml_client.environments.get(name=env_name, version="1")
    print("Found existing environment.")

except Exception as ex:
    #Print the error message
    print(ex)
    
    print("Creating new enviroment")
    env_docker_conda = Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        conda_file="../src/env.yml",
        name=env_name,
        description="Environment created from a Docker image plus Conda environment.",
    )
    env = ml_client.environments.create_or_update(env_docker_conda)

Found existing environment.


## 1.1.5 Prepare Example Data

In [7]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

order_path = "../sample_data/order_large.csv"
distances_path = "../sample_data/distance.csv"
# set the version number of the data asset
v1 = "initial"

order_data = Data(
    name="order-data",
    version=v1,
    description="Example order data",
    path=order_path,
    type=AssetTypes.URI_FILE,
)

distances_data = Data(
    name="distances-data",
    version=v1,
    description="Example distance data",
    path=distances_path,
    type=AssetTypes.URI_FILE,
)

## create order data asset if it doesn't already exist:
try:
    order_data_asset = ml_client.data.get(name=order_data.name, version=order_data.version)
    print(
        f"Data asset already exists. Name: {order_data.name}, version: {order_data.version}"
    )
except:
    ml_client.data.create_or_update(order_data)
    print(f"Data asset created. Name: {order_data.name}, version: {order_data.version}")

## create distances data asset if it doesn't already exist:
try:
    distances_data_asset = ml_client.data.get(name=distances_data.name, version=distances_data.version)
    print(
        f"Data asset already exists. Name: {distances_data.name}, version: {distances_data.version}"
    )
except:
    ml_client.data.create_or_update(distances_data)
    print(f"Data asset created. Name: {distances_data.name}, version: {distances_data.version}")


Data asset already exists. Name: order-data, version: initial
Data asset already exists. Name: distances-data, version: initial


# 1.2 Set up Azure ML Pipeline

This section contains the main logic of the optimization pipeline.

## 1.2.1 Reduce the search space of the problem

The first step is to reduce the search space by assigning some of the orders based on heuristic. The detailed logic is implemented in the reduce.py. In general, if we use heuristic propoerly, we can achieve a good trade-off between result optimality and running time.

In [ ]:
# import the components as functions
from src.reduce import reduce_component

cluster_name = "cpu_compute_target"
# define a pipeline with component
@pipeline(default_compute=cluster_name)
def pipeline_with_python_function_components(model_input, distance):
    """E2E dummy train-score-eval pipeline with components defined via Python function components"""

    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    reduce_step = reduce_component(
        model_input=model_input, distance=distance, learning_rate=learning_rate
    )

    # Return: pipeline outputs
    return {
    }


pipeline_job = pipeline_with_python_function_components(
    input_data=Input(
        path="wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv", type="uri_file"
    ),
    test_data=Input(
        path="wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv", type="uri_file"
    ),
    learning_rate=0.1,
)

# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="train_score_eval_pipeline"
)

# 1.2.2 Partition the problem

For large scale optimization problem, the problem space is just so big to solve practically. A commonly used idea is to partition the big problem into many smaller problems. Then solve each smaller problem individually and combine all the results as the final result. In some cases, the partition may not affect the result optimality, for example, in the route optimization problem, we can partition the orders by the delivery sources. In other cases, there will be trade-off between result optimality and running time when partitioning is applied. 

In [ ]:
# Naming the intermediate data 
model_input_list = PipelineData("model_input_list",datastore=def_blob_store).as_dataset()

parition_step = PythonScriptStep(
    script_name="partition.py", 
    arguments=["--model_input_reduced", model_input_reduced,
                "--distance", distance.as_named_input('distance').as_download(path_on_compute='distance_file'),
                "--model_input_list", model_input_list],
    inputs=[model_input_reduced],
    outputs=[model_input_list],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

## 1.2.3 Solve individual problem

After the problem is partitioned, we can solve each individul one by using whatever optimization solver. The optimization solver itself may leverage multi-process to speed up the search of result. This level of parallelism is totally controlled by the solver but not our Azure ML pipeline.

In [ ]:
import uuid

# Naming the intermediate data 
model_result_list = PipelineData("model_result_list", datastore=def_blob_store)

# pass distance file as side input
local_path = "/tmp/{}".format(str(uuid.uuid4()))
distance_config = distance.as_named_input("distance").as_mount(local_path)


parallel_run_config = ParallelRunConfig(
    source_directory=source_directory,
    entry_script='solve.py',
    mini_batch_size="5",
    error_threshold=-1,
    output_action="append_row",
    append_row_file_name="model_result_list.txt",
    environment=env,
    compute_target=aml_compute,
    run_invocation_timeout=600,
    node_count=max_nodes)

solve_step = ParallelRunStep(
    name="solve",
    inputs=[model_input_list.as_named_input('model_input_list')],
    output=model_result_list,
    arguments=["--distance", distance_config],
    side_inputs=[distance_config],
    parallel_run_config=parallel_run_config,
    allow_reuse=False
)

## 1.2.4 Merge the results

Once all the smaller problems are solved, we can combine the result as the final one. There could be chance to further optimize the result in this step in the case the previous partitioning will affect the global optimal. For example, one may combine two packages into the same truck from two seperated result if the combined one is more cost-efficient. 

In [ ]:
# Please replace it with the path you uploaded to Azure ML default datestore.
model_output_path = 'model_output'

# Naming the intermediate data 
model_result_final = OutputFileDatasetConfig(destination=(def_blob_store, model_output_path))

merge_step = PythonScriptStep(
    script_name="merge.py", 
    arguments=["--model_input", model_input.as_named_input('model_input').as_download(path_on_compute='order_file'), 
    "--distance", distance.as_named_input('distance').as_download(path_on_compute='distance_file'),
    "--model_result_partial", model_result_partial, 
    "--model_result_list", model_result_list, 
    "--model_result_final", model_result_final],
    inputs=[model_result_partial, model_result_list],
    outputs=[model_result_final],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

## 1.2.5 Run the Pipeline

Finally, we chained all steps into a single Azure ML pipeline and submit it to run.

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[reduce_step, parition_step, solve_step, merge_step])

print("Pipeline is built")

pipeline_run = Experiment(ws, 'optimization_example').submit(pipeline)
print("Pipeline is submitted for execution")

# RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion(show_output=True)

# 1.3 Check the Model Result

In [ ]:
model_output = Dataset.Tabular.from_delimited_files(path=(def_blob_store, model_output_path))

print(model_output.to_pandas_dataframe())

# 1.4 Publish the Pipeline

In [ ]:
# Publish the pipeline to Azure ML 

published_pipeline = pipeline_run.publish_pipeline(
    name='Route Optimization', description='Demo for route optimization', version='1.0')

published_pipeline

In [ ]:
# Print the endpoint of the pipeline

rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)